In [152]:
import numpy as np
import math

class InfGraph:
    
    def __init__(self, num_agts = 0):
        self._num_agts = num_agts
        self._bel = np.zeros(num_agts)
        self._graph = np.zeros((num_agts,num_agts))
        
    def change_edge(self, i, j, inf):
        self._graph[i][j] = inf
        
    def build_graph(self, graph_type="clique", inf = [0.5,0.1,0.6,0.8,0.2]):
        n = self._num_agts
        if(graph_type == "clique"):
            for i in range(n):
                for j in range(n):
                    self.change_edge(i,j,inf[0])
                    
        elif(graph_type == "disconnected"):
            for i in range(n):
                for j in range(n):
                    if(i+1 <= math.ceil(n/2) and j+1 <= math.ceil(n/2) or
                       i+1 > math.ceil(n/2) and j+1 > math.ceil(n/2)):
                        self.change_edge(i, j, inf[0])
                    else:
                        self.change_edge(i, j, 0)
                
        elif(graph_type == "faintly-connected"):
            for i in range(n):
                for j in range(n):
                    if(i+1 <= math.ceil(n/2) and j+1 <= math.ceil(n/2) or
                       i+1 > math.ceil(n/2) and j+1 > math.ceil(n/2)):
                        self.change_edge(i, j, inf[0])
                    else:
                        self.change_edge(i, j, inf[1])
                        
        elif(graph_type == "unrelenting-influencers"):
            # num_agts must be at least 2
            # inf1 = agt(0) and inf2 = agt(n-1)
            for i in range(n):
                for j in range(n):
                    if((i == 0 and j != n-1) or (i == n-1 and j!=0)):
                        self.change_edge(i, j, inf[2])
                    elif(i!=j and (j == 0 or j == n-1)):
                        self.change_edge(i, j, 0)
                    elif(0!=i and i!=n-1 and 0!=j and j!=n-1):
                        self.change_edge(i, j, inf[1])
                        
        elif(graph_type == "malleable-influencers"):
            # num_agts must be at least 2
            # inf1 = agt(0) and inf2 = agt(n-1)
            for i in range(n):
                for j in range(n):
                    if(i == 0 and j != n-1):
                        self.change_edge(i, j, inf[3])
                    elif(i!=n-1 and j!= 0):
                        self.change_edge(i, j, inf[0])
                    elif(i!=j and (j == 0 or j == n-1)):
                        self.change_edge(i, j, inf[1])
                    elif(0!=i and i!=n-1 and 0!=j and j!=n-1):
                        self.change_edge(i, j, inf[4])
                        
    def build_bel(self, bel_type="uniform", inf=[0, 1, 0.5]):
        n = self._num_agts
        if(bel_type == "uniform"):
            for i in range(self._num_agts):
                self._bel[i] = i/(n-1)
                    
        elif(bel_type == "mildly-polarized"):
            for i in range(n):
                if(i <= math.ceil(n/2)):
                    self._bel[i] = max(0.25-0.01*(math.ceil(n/2)-i-1), 0)
                else:
                    self._bel[i] = min(0.75-0.01*(math.ceil(n/2)-i),1)
            
        elif(bel_type == "extremely-polarized"):
            for i in range(n):
                if(i < math.ceil(n/2)): #I put < instead of <= because it made more sense in the case n is odd
                    self._bel[i] = inf[0]
                else:
                    self._bel[i] = inf[1]
                    
        elif(bel_type == "trypartite"):
            for i in range(n):
                if(i < math.floor(n/3)):
                    self._bel[i] = inf[0]
                elif (i < math.ceil(2*(n/3))):
                    self._bel[i] = inf[1]
                else:
                    self._bel[i] = inf[2]
            
    def print_graph(self):
        for i in range(self._num_agts):
            for j in range(self._num_agts):
                print(self._graph[i][j], end = " ")
            print()
    
    def print_bel(self):
        for i in self._bel:
            print(i, end = " ")
        print()

In [153]:
graph = InfGraph(num_agts = 10)

graph.build_bel(bel_type="uniform")
graph.print_bel()
graph.build_bel(bel_type="mildly-polarized")
graph.print_bel()
graph.build_bel(bel_type="extremely-polarized")
graph.print_bel()
graph.build_bel(bel_type="trypartite")
graph.print_bel()

0.0 0.111111111111 0.222222222222 0.333333333333 0.444444444444 0.555555555556 0.666666666667 0.777777777778 0.888888888889 1.0 
0.21 0.22 0.23 0.24 0.25 0.26 0.76 0.77 0.78 0.79 
0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 
0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.5 0.5 0.5 


In [154]:
graph.build_graph(graph_type="clique")
graph.print_graph()
print()
graph.build_graph(graph_type="disconnected")
graph.print_graph()
print()
graph.build_graph(graph_type="faintly-connected")
graph.print_graph()
print()
graph.build_graph(graph_type="unrelenting-influencers")
graph.print_graph()
print()
graph.build_graph(graph_type="malleable-influencers")
graph.print_graph()
print()

0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 
0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 

0.5 0.5 0.5 0.5 0.5 0.0 0.0 0.0 0.0 0.0 
0.5 0.5 0.5 0.5 0.5 0.0 0.0 0.0 0.0 0.0 
0.5 0.5 0.5 0.5 0.5 0.0 0.0 0.0 0.0 0.0 
0.5 0.5 0.5 0.5 0.5 0.0 0.0 0.0 0.0 0.0 
0.5 0.5 0.5 0.5 0.5 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.5 0.5 0.5 0.5 0.5 
0.0 0.0 0.0 0.0 0.0 0.5 0.5 0.5 0.5 0.5 
0.0 0.0 0.0 0.0 0.0 0.5 0.5 0.5 0.5 0.5 
0.0 0.0 0.0 0.0 0.0 0.5 0.5 0.5 0.5 0.5 
0.0 0.0 0.0 0.0 0.0 0.5 0.5 0.5 0.5 0.5 

0.5 0.5 0.5 0.5 0.5 0.1 0.1 0.1 0.1 0.1 
0.5 0.5 0.5 0.5 0.5 0.1 0.1 0.1 0.1 0.1 
0.5 0.5 0.5 0.5 0.5 0.1 0.1 0.1 0.1 0.1 
0.5 0.5 0.5 0.5 0.5 0.1 0.1 0.1 0.1 0.1 
0.5 0.5 0.5 0.